In [31]:
import os

from dotenv import load_dotenv
from openai import OpenAI

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

client = OpenAI(api_key=OPENAI_API_KEY)

In [32]:
import requests

SYSTEM_PROMPT = f"""당신의 이름은 '메뉴뚝딱AI'이며 당신의 역할은 배달의민족이라는 음식 주문 모바일 어플에서 리뷰 텍스트 기반으로 메뉴를 추천해주는 것입니다.
실제 배달의민족 어플 내 주문이 가능한 메뉴 및 음식점을 추천해줘야 하며 단순한 메뉴명을 추천해줄 수 없습니다. (ex. 해장국, 파스타)
추천 가능한 메뉴는 recommend 함수를 통해 결과를 받아 올 수 있습니다.
당신은 사용자의 발화를 기반으로 메뉴 추천 API를 호출하여 API 결과를 기반으로 사용자에게 최상의 추천 결과를 제공해야 합니다.
"""

MESSAGES = [
    {
        'role': 'system',
        'content': SYSTEM_PROMPT
    }
]

def recommend(query_text):
    url = "http://localhost:8000/recommend"
    response = requests.post(url, json={"query_text": query_text})
    return response.json()


tools = [
    {
        "type": "function",
        "function": {
            "name": "recommend",
            "description": "사용자 발화 기반으로 메뉴 추천 API를 호출합니다. 오로지 이 함수 결과로만 메뉴 추천되어야 합니다. 현재 해장 또는 다이어트 2개 카테고리에 대한 메뉴 추천만 가능하며, 사용자 발화가 없는 경우 빈 리스트가 반환 될 수 있습니다.",
            "parameters": {
                "type": "object",
                "properties": {
                    "query_text": {
                        "type": "string",
                        "description": "사용자 발화 텍스트 원문",
                    },
                },
                "required": ["query_text"],
                "additionalProperties": False,
            },
        }
    }
]

In [ ]:
MESSAGES.append(
    {"role": "user", "content": "안녕"}
)

completion = client.chat.completions.create(
    messages=MESSAGES,
    model="gpt-4o-2024-08-06",
    tools=tools,
    tool_choice="auto"
)

completion

ChatCompletion(id='chatcmpl-B0f3GneGRRB2tBG3E1I2eMQrhXbQ4', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='안녕하세요! 메뉴 추천이 필요하신가요? 어떤 종류의 음식을 드시고 싶으신지 말씀해주시면 추천해드릴게요.', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))], created=1739497502, model='gpt-4o-2024-08-06', object='chat.completion', service_tier='default', system_fingerprint='fp_523b9b6e5f', usage=CompletionUsage(completion_tokens=37, prompt_tokens=244, total_tokens=281, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))

In [34]:
if completion.choices[0].finish_reason == "tool_calls":
    print('function calling lets go')
else:
    MESSAGES.append(
        {"role": "assistant", "content": completion.choices[0].message.content}
    )

In [35]:
MESSAGES

[{'role': 'system',
  'content': "당신의 이름은 '메뉴뚝딱AI'이며 당신의 역할은 배달의민족이라는 음식 주문 모바일 어플에서 리뷰 텍스트 기반으로 메뉴를 추천해주는 것입니다.\n실제 배달의민족 어플 내 주문이 가능한 메뉴 및 음식점을 추천해줘야 하며 단순한 메뉴명을 추천해줄 수 없습니다. (ex. 해장국, 파스타)\n추천 가능한 메뉴는 recommend 함수를 통해 결과를 받아 올 수 있습니다.\n당신은 사용자의 발화를 기반으로 메뉴 추천 API를 호출하여 API 결과를 기반으로 사용자에게 최상의 추천 결과를 제공해야 합니다.\n"},
 {'role': 'user', 'content': '안녕'},
 {'role': 'assistant',
  'content': '안녕하세요! 메뉴 추천이 필요하신가요? 어떤 종류의 음식을 드시고 싶으신지 말씀해주시면 추천해드릴게요.'}]

In [ ]:
import json

MESSAGES.append(
    {"role": "user", "content": "해장 메뉴 추천해줄래?"}
)

completion = client.chat.completions.create(
    messages=MESSAGES,
    model="gpt-4o-2024-08-06",
    tools=tools,
    tool_choice="auto"
)

function calling lets go
[{'_id': '522757_0', 'score': 0.3734956060022517, 'menu': '달콤한나의고구마', 'restaurant': '청년피자-과천점', 'url': 'https://www.yogiyo.co.kr/mobile/#/522757/'}, {'_id': '522757_1', 'score': 0.2448980077371205, 'menu': '피자도우 변경(고구마체다크러스트)', 'restaurant': '청년피자-과천점', 'url': 'https://www.yogiyo.co.kr/mobile/#/522757/'}, {'_id': '1212106_0', 'score': 0.16673977787035865, 'menu': '바질 닭가슴살 랩', 'restaurant': 'SLB샐러드-과천점', 'url': 'https://www.yogiyo.co.kr/mobile/#/1212106/'}]


In [39]:
# ✅ tool_calls가 있는지 `completion.choices[0].message.tool_calls`에서 확인
if hasattr(completion.choices[0].message, "tool_calls") and completion.choices[0].message.tool_calls:
    tool_calls = completion.choices[0].message.tool_calls  # 🔥 수정된 부분
    tool_name = tool_calls[0].function.name
    tool_args = tool_calls[0].function.arguments
    tool_id = tool_calls[0].id
    print("function calling lets go")  # 🎯 이제 정상 출력될 것!

    # 🛠️ tool_args를 recommend() 함수에 전달
    tool_result = recommend(**json.loads(tool_args))
    print(tool_result[:3])

else:
    MESSAGES.append(
        {"role": "assistant", "content": completion.choices[0].message.content}
    )

function calling lets go
[{'_id': '522757_0', 'score': 0.37348178322077286, 'menu': '달콤한나의고구마', 'restaurant': '청년피자-과천점', 'url': 'https://www.yogiyo.co.kr/mobile/#/522757/'}, {'_id': '522757_1', 'score': 0.24487336857268738, 'menu': '피자도우 변경(고구마체다크러스트)', 'restaurant': '청년피자-과천점', 'url': 'https://www.yogiyo.co.kr/mobile/#/522757/'}, {'_id': '1212106_0', 'score': 0.16667210910478053, 'menu': '바질 닭가슴살 랩', 'restaurant': 'SLB샐러드-과천점', 'url': 'https://www.yogiyo.co.kr/mobile/#/1212106/'}]


In [40]:
completion.choices[0].message.tool_calls[0]

ChatCompletionMessageToolCall(id='call_iDCZkbK2SW7jMCQlP3hXkHAp', function=Function(arguments='{"query_text":"해장"}', name='recommend'), type='function')

In [41]:
tool_result = recommend(**json.loads(tool_args))
tool_result[:3]

[{'_id': '522757_0',
  'score': 0.37343329565378025,
  'menu': '달콤한나의고구마',
  'restaurant': '청년피자-과천점',
  'url': 'https://www.yogiyo.co.kr/mobile/#/522757/'},
 {'_id': '522757_1',
  'score': 0.24483591021498768,
  'menu': '피자도우 변경(고구마체다크러스트)',
  'restaurant': '청년피자-과천점',
  'url': 'https://www.yogiyo.co.kr/mobile/#/522757/'},
 {'_id': '1212106_0',
  'score': 0.16662085462838885,
  'menu': '바질 닭가슴살 랩',
  'restaurant': 'SLB샐러드-과천점',
  'url': 'https://www.yogiyo.co.kr/mobile/#/1212106/'}]

In [42]:
tool_calls[0].id

'call_iDCZkbK2SW7jMCQlP3hXkHAp'

In [43]:
MESSAGES

[{'role': 'system',
  'content': "당신의 이름은 '메뉴뚝딱AI'이며 당신의 역할은 배달의민족이라는 음식 주문 모바일 어플에서 리뷰 텍스트 기반으로 메뉴를 추천해주는 것입니다.\n실제 배달의민족 어플 내 주문이 가능한 메뉴 및 음식점을 추천해줘야 하며 단순한 메뉴명을 추천해줄 수 없습니다. (ex. 해장국, 파스타)\n추천 가능한 메뉴는 recommend 함수를 통해 결과를 받아 올 수 있습니다.\n당신은 사용자의 발화를 기반으로 메뉴 추천 API를 호출하여 API 결과를 기반으로 사용자에게 최상의 추천 결과를 제공해야 합니다.\n"},
 {'role': 'user', 'content': '안녕'},
 {'role': 'assistant',
  'content': '안녕하세요! 메뉴 추천이 필요하신가요? 어떤 종류의 음식을 드시고 싶으신지 말씀해주시면 추천해드릴게요.'},
 {'role': 'user', 'content': '해장 메뉴 추천해줄래?'},
 {'role': 'user', 'content': '해장 메뉴 추천해줄래?'},
 {'role': 'user', 'content': '해장 메뉴 추천해줄래?'}]

In [44]:
MESSAGES.append({
    "role": "assistant",  # user does not work
    "content": None,
    "tool_calls": [
        {
            "id": tool_id,
            "type": "function",
            "function": {
                "name": tool_name,
                "arguments": tool_args
            }
        }
    ]
})

In [45]:
tool_args

'{"query_text":"해장"}'

In [46]:
MESSAGES

[{'role': 'system',
  'content': "당신의 이름은 '메뉴뚝딱AI'이며 당신의 역할은 배달의민족이라는 음식 주문 모바일 어플에서 리뷰 텍스트 기반으로 메뉴를 추천해주는 것입니다.\n실제 배달의민족 어플 내 주문이 가능한 메뉴 및 음식점을 추천해줘야 하며 단순한 메뉴명을 추천해줄 수 없습니다. (ex. 해장국, 파스타)\n추천 가능한 메뉴는 recommend 함수를 통해 결과를 받아 올 수 있습니다.\n당신은 사용자의 발화를 기반으로 메뉴 추천 API를 호출하여 API 결과를 기반으로 사용자에게 최상의 추천 결과를 제공해야 합니다.\n"},
 {'role': 'user', 'content': '안녕'},
 {'role': 'assistant',
  'content': '안녕하세요! 메뉴 추천이 필요하신가요? 어떤 종류의 음식을 드시고 싶으신지 말씀해주시면 추천해드릴게요.'},
 {'role': 'user', 'content': '해장 메뉴 추천해줄래?'},
 {'role': 'user', 'content': '해장 메뉴 추천해줄래?'},
 {'role': 'user', 'content': '해장 메뉴 추천해줄래?'},
 {'role': 'assistant',
  'content': None,
  'tool_calls': [{'id': 'call_iDCZkbK2SW7jMCQlP3hXkHAp',
    'type': 'function',
    'function': {'name': 'recommend', 'arguments': '{"query_text":"해장"}'}}]}]

In [47]:
tool_result[:3]

[{'_id': '522757_0',
  'score': 0.37343329565378025,
  'menu': '달콤한나의고구마',
  'restaurant': '청년피자-과천점',
  'url': 'https://www.yogiyo.co.kr/mobile/#/522757/'},
 {'_id': '522757_1',
  'score': 0.24483591021498768,
  'menu': '피자도우 변경(고구마체다크러스트)',
  'restaurant': '청년피자-과천점',
  'url': 'https://www.yogiyo.co.kr/mobile/#/522757/'},
 {'_id': '1212106_0',
  'score': 0.16662085462838885,
  'menu': '바질 닭가슴살 랩',
  'restaurant': 'SLB샐러드-과천점',
  'url': 'https://www.yogiyo.co.kr/mobile/#/1212106/'}]

In [48]:
MESSAGES.append({
    "role": "tool",
    "content": json.dumps(tool_result[:3], ensure_ascii=False),
    "tool_call_id": tool_id
})

In [49]:
MESSAGES

[{'role': 'system',
  'content': "당신의 이름은 '메뉴뚝딱AI'이며 당신의 역할은 배달의민족이라는 음식 주문 모바일 어플에서 리뷰 텍스트 기반으로 메뉴를 추천해주는 것입니다.\n실제 배달의민족 어플 내 주문이 가능한 메뉴 및 음식점을 추천해줘야 하며 단순한 메뉴명을 추천해줄 수 없습니다. (ex. 해장국, 파스타)\n추천 가능한 메뉴는 recommend 함수를 통해 결과를 받아 올 수 있습니다.\n당신은 사용자의 발화를 기반으로 메뉴 추천 API를 호출하여 API 결과를 기반으로 사용자에게 최상의 추천 결과를 제공해야 합니다.\n"},
 {'role': 'user', 'content': '안녕'},
 {'role': 'assistant',
  'content': '안녕하세요! 메뉴 추천이 필요하신가요? 어떤 종류의 음식을 드시고 싶으신지 말씀해주시면 추천해드릴게요.'},
 {'role': 'user', 'content': '해장 메뉴 추천해줄래?'},
 {'role': 'user', 'content': '해장 메뉴 추천해줄래?'},
 {'role': 'user', 'content': '해장 메뉴 추천해줄래?'},
 {'role': 'assistant',
  'content': None,
  'tool_calls': [{'id': 'call_iDCZkbK2SW7jMCQlP3hXkHAp',
    'type': 'function',
    'function': {'name': 'recommend', 'arguments': '{"query_text":"해장"}'}}]},
 {'role': 'tool',
  'content': '[{"_id": "522757_0", "score": 0.37343329565378025, "menu": "달콤한나의고구마", "restaurant": "청년피자-과천점", "url": "https://www.yogiyo.co.kr/mobile/#/522757/"}, {"_id": "522757_1", "score"

In [50]:
completion = client.chat.completions.create(
    messages=MESSAGES,
    model="gpt-4o-2024-08-06",
    tools=tools
)
completion

ChatCompletion(id='chatcmpl-B0fB3EcSpg19QkLmkZss3onIWKTsf', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='해장을 위한 메뉴로 다음을 추천드립니다:\n\n1. [달콤한나의고구마](https://www.yogiyo.co.kr/mobile/#/522757/) - 청년피자-과천점\n2. [피자도우 변경(고구마체다크러스트)](https://www.yogiyo.co.kr/mobile/#/522757/) - 청년피자-과천점\n3. [바질 닭가슴살 랩](https://www.yogiyo.co.kr/mobile/#/1212106/) - SLB샐러드-과천점\n\n해당 링크를 통해 바로 주문이 가능하니 참고해 주세요! 요기요 어플을 통해 좀 더 다양한 선택지를 확인하실 수도 있습니다.', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))], created=1739497985, model='gpt-4o-2024-08-06', object='chat.completion', service_tier='default', system_fingerprint='fp_523b9b6e5f', usage=CompletionUsage(completion_tokens=165, prompt_tokens=560, total_tokens=725, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0

In [51]:
MESSAGES.append(
    {"role": "assistant", "content": completion.choices[0].message.content}
)

In [52]:
MESSAGES

[{'role': 'system',
  'content': "당신의 이름은 '메뉴뚝딱AI'이며 당신의 역할은 배달의민족이라는 음식 주문 모바일 어플에서 리뷰 텍스트 기반으로 메뉴를 추천해주는 것입니다.\n실제 배달의민족 어플 내 주문이 가능한 메뉴 및 음식점을 추천해줘야 하며 단순한 메뉴명을 추천해줄 수 없습니다. (ex. 해장국, 파스타)\n추천 가능한 메뉴는 recommend 함수를 통해 결과를 받아 올 수 있습니다.\n당신은 사용자의 발화를 기반으로 메뉴 추천 API를 호출하여 API 결과를 기반으로 사용자에게 최상의 추천 결과를 제공해야 합니다.\n"},
 {'role': 'user', 'content': '안녕'},
 {'role': 'assistant',
  'content': '안녕하세요! 메뉴 추천이 필요하신가요? 어떤 종류의 음식을 드시고 싶으신지 말씀해주시면 추천해드릴게요.'},
 {'role': 'user', 'content': '해장 메뉴 추천해줄래?'},
 {'role': 'user', 'content': '해장 메뉴 추천해줄래?'},
 {'role': 'user', 'content': '해장 메뉴 추천해줄래?'},
 {'role': 'assistant',
  'content': None,
  'tool_calls': [{'id': 'call_iDCZkbK2SW7jMCQlP3hXkHAp',
    'type': 'function',
    'function': {'name': 'recommend', 'arguments': '{"query_text":"해장"}'}}]},
 {'role': 'tool',
  'content': '[{"_id": "522757_0", "score": 0.37343329565378025, "menu": "달콤한나의고구마", "restaurant": "청년피자-과천점", "url": "https://www.yogiyo.co.kr/mobile/#/522757/"}, {"_id": "522757_1", "score"